In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,058 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,776 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http://

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import year

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [5]:
# Show schema to confirm date type
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [6]:
# Show the year for the date column
df.select(year(df["date"])).show()

+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2019|
|      2019|
|      2022|
|      2019|
|      2020|
|      2019|
|      2020|
|      2021|
|      2019|
|      2021|
|      2020|
|      2021|
|      2019|
|      2021|
|      2021|
|      2019|
|      2019|
|      2019|
+----------+
only showing top 20 rows



In [7]:
# Save the year as a new column
df = df.withColumn("year", year(df['date']))
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
    SELECT YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
    ORDER BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
""").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
    SELECT YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
    ORDER BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
""").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|    287287.82|
|2020|    294204.16|
|2021|    294211.46|
|2022|    292725.69|
+----+-------------+



In [10]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""
    SELECT YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
    ORDER BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
""").show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|    289859.14|
|2020|    292289.09|
|2021|    296330.96|
|2022|    290242.99|
+----+-------------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
    SELECT YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS year, ROUND(AVG(price), 2) AS average_price, MAX(view) AS view_rating
    FROM home_sales
    WHERE price >= 350000
    GROUP BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
    ORDER BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+-----------+
|year|average_price|view_rating|
+----+-------------+-----------+
|2019|    478628.66|         99|
|2020|    476026.01|         99|
|2021|    468070.46|         99|
|2022|    471010.02|         99|
+----+-------------+-----------+

--- 1.2224106788635254 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
# Query using the cached table
result = spark.sql("""
    SELECT YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS year, ROUND(AVG(price), 2) AS average_price, MAX(view) AS view_rating
    FROM home_sales
    WHERE price >= 350000
    GROUP BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
    ORDER BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+-----------+
|year|average_price|view_rating|
+----+-------------+-----------+
|2019|    478628.66|         99|
|2020|    476026.01|         99|
|2021|    468070.46|         99|
|2022|    471010.02|         99|
+----+-------------+-----------+

--- 0.7368068695068359 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.parquet('date_built', mode='overwrite')

In [16]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('date_built')

In [17]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('date_built')

In [18]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
spark.sql("""
    SELECT YEAR(TO_DATE(date, 'yyyy-MM-dd')) AS year, ROUND(AVG(price), 2) AS average_price, MAX(view) AS view_rating
    FROM home_sales
    WHERE price >= 350000
    GROUP BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
    ORDER BY YEAR(TO_DATE(date, 'yyyy-MM-dd'))
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+-----------+
|year|average_price|view_rating|
+----+-------------+-----------+
|2019|    478628.66|         99|
|2020|    476026.01|         99|
|2021|    468070.46|         99|
|2022|    471010.02|         99|
+----+-------------+-----------+

--- 0.8409497737884521 seconds ---


In [19]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [20]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales')

if is_cached:
    print("The table is still cached.")
else:
    print("The table is not cached.")

The table is not cached.
